In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/477

/content/drive/MyDrive/477


In [ ]:

def Load_data(filename):
    with open(filename) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    Transaction = []
    for i in range(0, len(content)):
        Transaction.append(content[i].split())
    return Transaction

def create_initialset(dataset):  # duplicate rows counting
    retDict = {}

    for trans in dataset:
        #print(trans)
        retDict[tuple(trans)] = 1
    #print(retDict)
    length=len(retDict)
    return length,retDict



In [ ]:
import time
from collections import defaultdict
import os,psutil
from memory_profiler import profile
pid=os.getpid()
ps=psutil.Process(pid)

#class of FP TREE node


class TreeNode:
    def __init__(self, Node_name,counter,parentNode):
        self.name = Node_name
        self.count = counter
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
    def increment_counter(self, counter):
        self.count += counter

def create_FPTree(dataset, minSupport): #main task
    HeaderTable = {} #header table

    for transaction in dataset:
        for item in transaction: #1 2 3
            HeaderTable[item] = HeaderTable.get(item,0) + dataset[transaction]
            #print(dataset[transaction],"hel")
    #print(HeaderTable)
    for k in list(HeaderTable):
        if HeaderTable[k] < minSupport:
            del(HeaderTable[k])

    frequent_itemset = set(HeaderTable.keys())

    if len(frequent_itemset) == 0:
        return None, None

    for k in HeaderTable:
        HeaderTable[k] = [HeaderTable[k], None]
     # 5 - [5,None]

    retTree = TreeNode('Null Set',1,None) # Main root Tree

    for itemset,count in dataset.items():
        frequent_transaction = {}

        for item in itemset:  #header table wise sorting kori
            if item in frequent_itemset:
                frequent_transaction[item] = HeaderTable[item][0]
                #3 5 = 5
        if len(frequent_transaction) > 0:
            ordered_itemset = []
            sorted_items = sorted(frequent_transaction.items(), key=lambda p: p[1], reverse=True)
            #print(sorted_items)
            for v in sorted_items:
              ordered_itemset.append(v[0])
            #to update the FPTree
            updateTree(ordered_itemset, retTree, HeaderTable, count) #row by row updating
    return retTree, HeaderTable


def updateTree(itemset, FPTree, HeaderTable, count):

    if itemset[0] in FPTree.children:
        FPTree.children[itemset[0]].increment_counter(count)
    else:
        new_node = TreeNode(itemset[0], count, FPTree) # i1 i2 i3
        FPTree.children[itemset[0]] = new_node

        if HeaderTable[itemset[0]][1] is None:
            HeaderTable[itemset[0]][1] = new_node
        else:
            update_NodeLink(HeaderTable[itemset[0]][1], new_node)
    if len(itemset) > 1:
        updateTree(itemset[1:], FPTree.children[itemset[0]], HeaderTable, count)


def update_NodeLink(Test_Node, Target_Node):
    while (Test_Node.nodeLink != None):
        Test_Node = Test_Node.nodeLink
    Test_Node.nodeLink = Target_Node


def FPTree_uptransveral(leaf_Node, prefixPath):
 if leaf_Node.parent != None:
    prefixPath.append(leaf_Node.name)
    FPTree_uptransveral(leaf_Node.parent, prefixPath)


def prefix_path(basePat, TreeNode):
    Conditional_patterns_base = {}

    while TreeNode is not None:
        prefixPath = []
        FPTree_uptransveral(TreeNode, prefixPath)
        if len(prefixPath) > 1:
            conditional_set = set(prefixPath[1:])
            Conditional_patterns_base[tuple(conditional_set)] = TreeNode.count
            #print(TreeNode.name,TreeNode.count,'helo')
        TreeNode = TreeNode.nodeLink

    return Conditional_patterns_base


def Mine_Tree(FPTree, HeaderTable, minSupport, prefix, frequent_itemset):
    bigL = []
    pettern_len=[]
    pairs = HeaderTable.items()
    pattern_lengths=[]


    sorted_pairs = sorted(pairs, key=lambda p: p[1][0])

    for v in sorted_pairs:
      key, _ = v

      bigL.append(key)

    for basePat in bigL:  #
        new_frequentset = prefix.copy()
        new_frequentset.add(basePat)
        # Add frequent itemset to final list of frequent itemsets
        support = HeaderTable[basePat][1].count
        #print(support)
        pattern_lengths.append((new_frequentset, support))
       # print(new_frequentset)

        frequent_itemset.append(new_frequentset)
        #pattern_lengths.setdefault(len(new_frequentset), 0)


        conditional_base = prefix_path(basePat, HeaderTable[basePat][1])

        Conditional_FPTree, Conditional_header = create_FPTree(conditional_base, minSupport)

        if Conditional_header is not None:
            Mine_Tree(Conditional_FPTree, Conditional_header, minSupport, new_frequentset, frequent_itemset)
    return pattern_lengths

def print_tree2(node, indent='', last_child=False):
  if node is not None:
    node_str = f"{node.name} ({node.count})"
    branch = "└─ " if last_child else "├─ "
    print(indent + branch + node_str)
    indent += "    " if last_child else "│   "
    children = list(node.children.values())
    for i, child in enumerate(children):
      is_last = i == len(children) - 1
      print_tree2(child, indent, is_last)

def write_tree_to_file(node, file, indent='', last_child=False):
  if node is not None:
    node_str = f"{node.name} ({node.count})"
    branch = "└─ " if last_child else "├─ "
    file.write(indent + branch + node_str + '\n')
    indent += "    " if last_child else "│   "
    children = list(node.children.values())
    for i, child in enumerate(children):
      is_last = i == len(children) - 1
      write_tree_to_file(child, file, indent, is_last)

def write_tree(names):
  filename = names
  with open(filename, 'w') as file:
    write_tree_to_file(FPtree, file)

def print_support_counts(frequent_itemset, HeaderTable):
    for itemset in frequent_itemset:
        support_count = min(HeaderTable[item][0] for item in itemset)
        print(f"Support Count of {itemset}: {support_count}")


filename = "mushroom.dat"
write_file="output_tree.txt"
length,initSet = create_initialset(Load_data(filename))
min_Support = (60/100)*length

start = time.time()


FPtree, HeaderTable = create_FPTree(initSet, min_Support)
frequent_itemset = []
txt=Mine_Tree(FPtree, HeaderTable, min_Support, set([]), frequent_itemset)
end = time.time()

print("Time Taken:")
print(end-start)

print("Total Patterns" ,len(frequent_itemset))
print_tree2(FPtree)
write_tree(write_file)
print("All frequent itemsets:")
frequent_itemset = sorted(frequent_itemset, key=lambda x: len(x))

for i in frequent_itemset:
  print(i)

memoryUse=ps.memory_info()


Time Taken:
0.1286940574645996
Total Patterns 51
├─ Null Set (1)
│   └─ 85 (8124)
│       ├─ 86 (7924)
│       │   ├─ 34 (7906)
│       │   │   ├─ 90 (7288)
│       │   │   │   ├─ 36 (6272)
│       │   │   │   │   ├─ 59 (1360)
│       │   │   │   │   │   └─ 63 (880)
│       │   │   │   │   ├─ 39 (4016)
│       │   │   │   │   │   ├─ 59 (2576)
│       │   │   │   │   │   │   └─ 63 (2312)
│       │   │   │   │   │   └─ 63 (72)
│       │   │   │   │   └─ 63 (432)
│       │   │   │   ├─ 39 (768)
│       │   │   │   │   ├─ 59 (384)
│       │   │   │   │   │   └─ 63 (192)
│       │   │   │   │   └─ 63 (192)
│       │   │   │   ├─ 59 (224)
│       │   │   │   │   └─ 63 (212)
│       │   │   │   └─ 63 (12)
│       │   │   ├─ 36 (330)
│       │   │   │   └─ 39 (330)
│       │   │   │       └─ 59 (296)
│       │   │   │           └─ 63 (296)
│       │   │   └─ 39 (288)
│       │   │       ├─ 59 (144)
│       │   │       │   └─ 63 (72)
│       │   │       └─ 63 (72)
│       │   └─ 36 (18)
│      